In [ ]:
#imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import time

from scipy.stats import qmc

mpl.rcParams["font.family"] = "Times New Roman"
mpl.rcParams["font.size"] = 12
mpl.rcParams["text.usetex"] = True

In [ ]:
tf.keras.backend.set_floatx("float32")

In [ ]:
n_bc = 4
n_data_per_bc = 8

engine = qmc.LatinHypercube(d=1)
data = np.zeros([n_bc, n_data_per_bc, 3])

In [ ]:
for i, j in zip(range(n_bc), [-1, +1, -1, +1]):
    points = (engine.random(n=n_data_per_bc)[:, 0] - 0.5) * 2
    if i < 2:
        data[i,:,0] = j
        data[i,:,1] = points
    else:
        data[i,:,0] = points
        data[i,:,1] = j

In [ ]:
data[1,:,2] = 1.
data[2,:,2] = 1.
data[3,:,2] = 1.

In [ ]:
data = data.reshape(n_data_per_bc*n_bc, 3)

In [ ]:
#
x_d, y_d, t_d = map(lambda x: np.expand_dims(x, axis=1), 
                    [data[:, 0], data[:, 1], data[:, 2]])

In [ ]:
#
N_c = 10000
engine = qmc.LatinHypercube(d=2)
colloc = engine.random(n=N_c)
colloc = 2 * (colloc -0.5)

In [ ]:
x_c, y_c = map(lambda x: np.expand_dims(x, axis=1), 
               [colloc[:, 0], colloc[:, 1]])

In [ ]:
# Transform to float32
x_c, y_c, x_d, y_d, t_d =map(lambda x: tf.convert_to_tensor(x,dtype=tf.float32),
                             [x_c, y_c, x_d, y_d, t_d])

In [ ]:
plt.figure("", figsize=(6, 6))
plt.title("Boundary Data points and Collocation points")

plt.scatter(x_d, y_d, marker='x', c='k', label='BDP')
plt.scatter(x_c, y_c, s=.2, marker=".", c="r", label="CP")
plt.show()

In [ ]:
### model builder function
def DNN_builder(in_shape=2, out_shape=1, n_hidden_layers=10, 
                neuron_per_layer=20, actfn="tanh"):
    # input layer
    input_layer = tf.keras.layers.Input(shape=(in_shape,))
    # hidden layers
    hidden = [tf.keras.layers.Dense(neuron_per_layer, activation=actfn)(input_layer)]
    for i in range(n_hidden_layers-1):
        new_layer = tf.keras.layers.Dense(neuron_per_layer,
                                          activation=actfn,
                                          activity_regularizer=None)(hidden[-1])
        hidden.append(new_layer)
    # output layer
    output_layer = tf.keras.layers.Dense(1, activation=None)(hidden[-1])
    # building the model
    name = f"DNN-{n_hidden_layers}"
    model = tf.keras.Model(input_layer, output_layer, name=name)
    return model

In [ ]:
layers = [1, 4, 8]
loss_dict = {}
train_time = {}

n = 100
X = np.linspace(-1, +1, n)
Y = np.linspace(-1, +1, n)
X0, Y0 = np.meshgrid(X, Y)
X = X0.reshape([n*n, 1])
Y = Y0.reshape([n*n, 1])
X_T = tf.convert_to_tensor(X)
Y_T = tf.convert_to_tensor(Y)


for it, layer in enumerate(layers):
    # os.mkdir(f"outputs{layer}")
    
    @tf.function
    def f(x, y):
        u0 = u(x, y)
        u_x = tf.gradients(u0, x)[0]
        u_y = tf.gradients(u0, y)[0]
        u_xx = tf.gradients(u_x, x)[0]
        u_yy = tf.gradients(u_y, y)[0]
        F = u_xx + u_yy
        return tf.reduce_mean(tf.square(F))
    
    @tf.function
    def u(x, y):
        u = model(tf.concat([x, y], axis=1))
        return u
    @tf.function
    def mse(y, y_):
        return tf.reduce_mean(tf.square(y-y_))

    
    tf.keras.backend.clear_session()
    model = DNN_builder(2, 1, layer, 20, "tanh")
    # model.summary()
    # tf.keras.utils.plot_model(model, to_file='model_plot.png', show_shapes=True, 
    #                           show_layer_names=True, show_dtype=True, 
    #                           show_layer_activations=True)
    
    loss = 0
    epochs = 1000
    opt = tf.keras.optimizers.legacy.Adam(learning_rate=1e-3)

    
    epoch = 0
    loss_values = np.array([])
    start = time.time()
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            T_ = u(x_d, y_d)
            l = mse(t_d, T_)
    
            L = f(x_c, y_c)
            loss = l+L
        g = tape.gradient(loss, model.trainable_weights)
        opt.apply_gradients(zip(g, model.trainable_weights))
        loss_values = np.append(loss_values, loss)
        if epoch % 20 == 0 or epoch == epochs-1:
            print(f"{epoch:5}, {loss.numpy():.3f}")
            S = u(X_T, Y_T)*10.
            S = S.numpy().reshape(n, n)
            np.savetxt(f"outputs{layer}/outputPINN{epoch // 20+1}.dat", S, fmt="%lf")
            
    end = time.time()
    print(f"\ncomputation time: {end-start:.3f}\n")

    loss_dict[layer] = loss_values
    train_time[layer] = end-start

In [ ]:
fig = plt.figure(figsize=(6,5), dpi=150)
colors = {1: "b", 4:"b" ,8: "r"}
linestypes = {1:"-", 4: "-.", 8: "-"}
labels = {1: "PINN-$1$", 
          4:"PINN-$4$", 
          8:"PINN-$8$"} 
# xvalues = df[df.columns[0]]

for it in layers:
    # it = df.columns[i]
    plt.semilogy(loss_dict[it], 
             label=labels[it], 
             color=colors[it],
             linestyle=linestypes[it]
            )
# plt.xlim([0,1e3])
plt.ylabel("Estimated Mean Square Errors ($MSE$)")
plt.xlabel("Number of Iterations")
plt.grid()
plt.legend()
plt.title("Traning Results of Heat Equation with $d=2$ by PINN")
plt.savefig("/Users/michaellea/001 - M.Sc_HPC/55640-Project/Seminar-2/Presentation/Slides/F0PINN.png", bbox_inches='tight')
plt.show()

In [ ]:
n = 100
# l = 1.
# r = 2*l/(n+1)
# T = np.zeros([n*n, n*n])

## FDM

In [ ]:
n = 100
l = 1.
r = 2*l/(n+1)
T = np.zeros([n*n, n*n])

bc = {
    "x=-l": 1,
    "x=+l": 0.,
    "y=-l": 1.,
    "y=+l": 1.
}

B = np.zeros([n, n])
k = 0
for i in range(n):
    x = i * r
    for j in range(n):
        y = j * r
        M = np.zeros([n, n])
        M[i, j] = -4
        if i != 0: # ok i know
            M[i-1, j] = 1
        else:
            B[i, j] += -bc["y=-l"]   # b.c y = 0
        if i != n-1:
            M[i+1, j] = 1
        else:
            B[i, j] += -bc["y=+l"]   # b.c y = l
        if j != 0:
            M[i, j-1] = 1
        else:
            B[i, j] += -bc["x=-l"]   # b.c x = 0
        if j != n-1:
            M[i, j+1] = 1
        else:
            B[i, j] += -bc["x=+l"]   # b.c x = l
        #B[i, j] += -r**2 * q(x, y) * K(x, y)
        m = np.reshape(M, (1, n**2))
        T[k, :] = m
        k += 1

#
b = np.reshape(B, (n**2, 1))
start = time.time()
T = np.matmul(np.linalg.inv(T), b)
T = T.reshape([n, n])
Temperature = T
end = time.time()
computation_time["fdm"] = end - start
print(f"\ncomputation time: {end-start:.3f}\n")

## Visualization

In [ ]:
plt.figure("", figsize=(8, 6),dpi=200)
#
X = np.linspace(-1, +1, n)
Y = np.linspace(-1, +1, n)
X0, Y0 = np.meshgrid(X, Y)
X = X0.reshape([n*n, 1])
Y = Y0.reshape([n*n, 1])
X_T = tf.convert_to_tensor(X)
Y_T = tf.convert_to_tensor(Y)

plt.subplot(221)
S = u(X_T, Y_T)
S = S.numpy().reshape(n, n)
plt.pcolormesh(-X0, Y0, 10.*S, cmap="jet")
plt.title("PINN 8 Hidden Layers")
plt.xlabel("$x$")
plt.ylabel("$y$")
plt.colorbar()

plt.subplot(222)
x = np.linspace(-1, +1, n)
y = np.linspace(-1, +1, n)
x, y = np.meshgrid(x, y)
plt.pcolormesh(x, y, T, cmap="jet")
plt.title("FDM")
plt.xlabel("$x$")
plt.ylabel("$y$")
plt.colorbar()


plt.subplot(223)
pinn_grad = np.gradient(np.gradient(S, axis=0), axis=1)
sigma_pinn = (pinn_grad**2).mean()
plt.pcolormesh(-X0, Y0, pinn_grad, cmap="jet")
plt.colorbar()
plt.xlabel("$x$")
plt.ylabel("$y$")
plt.title(f"\nmean squared: {sigma_pinn: .3e}")
plt.tight_layout()
plt.axis("square")
###
x = np.linspace(-1, +1, n)
y = np.linspace(-1, +1, n)
x, y = np.meshgrid(x, y)
#
plt.subplot(224)
fdm_grad = np.gradient(np.gradient(T, axis=0), axis=1)
sigma_fdm = (fdm_grad**2).mean()
plt.pcolormesh(x, y, fdm_grad, cmap="jet")
plt.colorbar()
plt.title(f"\nmean squared: {sigma_fdm: .3e}")
plt.xlabel("$x$")
plt.ylabel("$y$")
plt.xlim(-1, +1)
plt.ylim(-1, +1)
plt.tight_layout()
plt.axis("square")

plt.savefig(f"/Users/michaellea/001 - M.Sc_HPC/55640-Project/Seminar-2/Presentation/Slides/F0PINN_COMPARE.png", bbox_inches='tight')

plt.show()

In [ ]:
S

In [ ]:
np.savetxt("test.dat", S, fmt="%lf")